In [ ]:
import os
from dotenv import load_dotenv
import asyncio

from polymarket.src.stream.polymarket import WebSocketListener
from polymarket.src.stream.binance import binance_price_ws
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from polymarket.src.db.ingest_stream import insert_into_db, parse_message
from polymarket.src.config.logger import logger

In [ ]:
conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;'
)
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()
MARKET_CHANNEL = "market"
USER_CHANNEL = "user"
load_dotenv("../../.env")

url = os.getenv("POLYMARKET_WEBSOCKET_URL")

asset_ids = [
    "115172850081268399385975128465963399335156000901458077700537686348329692537170",
]


async def main():
    loop = asyncio.get_running_loop()
    listener = WebSocketListener(url, asset_ids, queue=queue, loop=loop, market_channel=MARKET_CHANNEL)
    executor = ProcessPoolExecutor(max_workers=10)
    producers = [
        asyncio.create_task(binance_price_ws(queue=queue, symbols=symbols)),
        asyncio.create_task(listener.run()),
    ]
    workers = [asyncio.create_task(insert_into_db(conf, table_name, queue, parse_message, executor, logger))]

    await asyncio.gather(*producers, *workers)

await main()

In [ ]:
url